In [61]:
### MERGE ######  
import os
import pandas as pd
import numpy as np

# Parent folder path
parent_folder = f'/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/proc_data/'
print('Folder:' + parent_folder)

# Parent folder path
concord_folder = f'/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/raw_data/'
print('Folder:' + concord_folder)

# merge master with state code and region
df_st = pd.read_excel(os.path.join(concord_folder, 'Concordance-Tables-Capex.xlsx'), sheet_name = 'state-concord') # state numeric code and region
df2 = pd.read_excel(os.path.join(parent_folder, 'Fiscal_Merged.xlsx'))
df2 = df2.rename(columns={'Years':'Year'})
df3 = pd.read_excel(os.path.join(parent_folder, 'Gross_Merged_LF.xlsx'))
df4 = pd.read_excel(os.path.join(parent_folder, 'Merged_Net_GDP.xlsx'))

dflist =[df2,df3,df4]

for i in dflist:
    i = i.rename(columns={'Year':'FY'}, inplace = True)

cc = ['State', 'FY']

# Merge Fiscal data
merged_fiscal = pd.merge(df2, df3, on=cc, how='inner').merge(df4, on = cc, how='inner').merge(
    df_st[['st_name_rbi','st_ut_code','region']], 
         left_on = ['State'], right_on = ['st_name_rbi'], how= 'left')

# Define the common pattern to remove
pattern1 = 'Gross State Value Added by Economic Activity - '
pattern2 = 'Net State Value Added by Economic Activity - '

# Rename columns using str.replace
merged_fiscal.columns = merged_fiscal.columns.str.replace(pattern1, 'GSVA-')
merged_fiscal.columns = merged_fiscal.columns.str.replace(pattern2, 'NSVA-')

#Split values based on hyphen and save the part before hyphen in a new column
merged_fiscal['FY'] = merged_fiscal['FY'].str.split('-').str[0].str[-2:] + '-' + merged_fiscal['FY'].str.split('-').str[1].str[-2:]

print(merged_fiscal.shape)

df1 = pd.read_excel(os.path.join(parent_folder, 'Capex-Projects-Total.xlsx'))

merged_capex_fiscal = pd.merge(df1, merged_fiscal, on=['st_ut_code','FY'], how='left')
print(merged_capex_fiscal.shape)


### Write Merged file to Excel 
out_folder_path = '/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/proc_data/'
out_file_name = 'Final-Capex.xlsx'
out_file_path = os.path.join(out_folder_path, out_file_name)

merged_capex_fiscal.to_excel((out_file_path), index=False)

print('FINITO')


Folder:/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/proc_data/
Folder:/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/raw_data/
(270, 30)
(81197, 56)
FINITO


In [55]:
import pandas as pd

# Sample data
data = {
    'Column_with_Hyphen': ['value-1', 'value-2', 'value-3']
}

# Create a DataFrame
df = pd.DataFrame(data)

# Split values based on hyphen and save the part before hyphen in a new column
df['after_Hyphen'] = df['Column_with_Hyphen'].str.split('-').str[1]

df['FY'] = df['Column_with_Hyphen'].str.split('-').str[0].str[-2:] + '-' + df['Column_with_Hyphen'].str.split('-').str[1].str[-2:]


print(df)


  Column_with_Hyphen after_Hyphen    FY
0            value-1            1  ue-1
1            value-2            2  ue-2
2            value-3            3  ue-3
